In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from pyopenms import *
import pandas as pd
import subprocess
import os

In [ ]:
# Define directories
raw_data_dir = "/home/vivian.chu/vivian-sandbox/neoantigens/ms_raw"
results_dir = "processed_results"

# Create results directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# List all files in the raw data directory
all_files = os.listdir(raw_data_dir)

# Filter for relevant .raw and .mgf files
raw_files = [f for f in all_files if f.endswith('.raw')]
mgf_files = [f for f in all_files if f.endswith('.mgf')]

In [ ]:
print("Raw Files:", raw_files)
print("MGF Files:", mgf_files)

In [ ]:
# Function to process raw files using PyOpenMS
def process_raw_data(file_path):
    """
    Process raw files using PyOpenMS.
    Extracts spectra and chromatographic data and returns as a DataFrame.
    """
    print(f"Processing raw file: {file_path}")

    # Load the raw data
    exp = MSExperiment()
    MzMLFile().load(file_path, exp)

    # Extract spectrum data
    spectra_data = []
    for spectrum in exp:
        if spectrum.getMSLevel() == 1:  # MS1 spectra (precursor ion scan)
            mz = spectrum.get_peaks()[0]  # Mass-to-charge ratios
            intensity = spectrum.get_peaks()[1]  # Intensities
            for m, i in zip(mz, intensity):
                spectra_data.append({"m/z": m, "intensity": i, "scan_index": spectrum.getNativeID()})
    
    # Convert to DataFrame
    df = pd.DataFrame(spectra_data)
    print(f"Extracted {len(df)} peaks from raw file: {file_path}")
    return df

In [ ]:
# Process the converted mzML file
raw_data = process_raw_data(mzml_file)
raw_data.head()

In [ ]:
# Function to process mgf files using PyOpenMS
def process_mgf_data(file_path):
    """
    Process MGF files using PyOpenMS.
    Extracts MS/MS data and returns as a DataFrame.
    """
    print(f"Processing MGF file: {file_path}")

    # Load the MGF data
    exp = MSExperiment()
    MzMLFile().load(file_path, exp)

    # Extract MS/MS data
    msms_data = []
    for spectrum in exp:
        if spectrum.getMSLevel() == 2:  # MS2 spectra (fragment ion scan)
            mz = spectrum.get_peaks()[0]  # Mass-to-charge ratios
            intensity = spectrum.get_peaks()[1]  # Intensities
            precursor_mz = spectrum.getPrecursors()[0].getMZ() if spectrum.getPrecursors() else None
            for m, i in zip(mz, intensity):
                msms_data.append({
                    "precursor_m/z": precursor_mz,
                    "fragment_m/z": m,
                    "intensity": i,
                    "scan_index": spectrum.getNativeID()
                })
    
    # Convert to DataFrame
    df = pd.DataFrame(msms_data)
    print(f"Extracted {len(df)} fragments from MGF file: {file_path}")
    return df

In [ ]:
# Example usage for a raw file
raw_file_path = "/home/vivian.chu/vivian-sandbox/neoantigens/ms_raw/B-ALL_MAE_fresh_2M_1.raw"
raw_data = process_raw_data(raw_file_path)
raw_data.head()
# raw_data.to_csv("processed_raw_data.csv", index=False)

In [ ]:
# Example usage for an mgf file
mgf_file_path = "/home/vivian.chu/vivian-sandbox/neoantigens/ms_raw/B-ALL_MAE_fresh_2M_1.mgf"
mgf_data = process_mgf_data(mgf_file_path)
mgf_data.to_csv("processed_mgf_data.csv", index=False)

In [ ]:
# Function to normalize peak areas
def normalize_peaks(df, condition_column="Condition", peak_column="PeakArea"):
    """
    Normalize peak areas by condition.
    """
    df[peak_column] = df.groupby(condition_column)[peak_column].apply(lambda x: x / x.median())
    return df

# Example criteria for filtering peptides
criteria = {
    "fdr": 0.05,
    "min_length": 8,
    "max_length": 15,
    "maf": 0.05
}

# Function to filter peptides
def filter_peptides(peptides_df, criteria):
    """
    Filter peptides based on given criteria.
    """
    filtered_peptides = peptides_df[
        (peptides_df['FDR'] <= criteria['fdr']) &
        (peptides_df['Length'].between(criteria['min_length'], criteria['max_length'])) &
        (peptides_df['MAF'] >= criteria['maf'])
    ]
    return filtered_peptides

# Example visualization for peptide overlap
def visualize_peptide_overlap(set1, set2, label1, label2):
    """
    Visualize overlap between two peptide sets.
    """
    venn = venn2([set(set1), set(set2)], (label1, label2))
    plt.title("Peptide Overlap")
    plt.show()

# Example usage of visualization
# (Replace with real peptide sets)
visualize_peptide_overlap(["peptide1", "peptide2"], ["peptide2", "peptide3"], "Set A", "Set B")

print("Processing complete. Results are saved in", results_dir)